In [ ]:
import os
import sys
from pathlib import Path

PROJ_ROOT = Path.cwd().parent
if str(PROJ_ROOT) not in sys.path:
    sys.path.append(str(PROJ_ROOT))

import pandas as pd
import numpy as np
import pydicom
pydicom.config.settings.reading_validation_mode = pydicom.config.IGNORE

import plistlib

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from ipywidgets import interact

from utils.functions import load_config, get_plist_schema
from dataset.common import load_patient_data, load_scan
from dataset.process import GeneratePatientData, load_clean_xml

In [ ]:
cfg = load_config()

# Make Patient Image-XML Matching Table

## Dicom Files

### Generated Strucutred Data from DICOM

In [ ]:
# Run this cell to generate patient data
# patient_df = GeneratePatientData(cfg.paths.raw_data, saveFileName='patient_data_w_metadata.csv')

In [ ]:
# Load Patient Data
patient_df = load_patient_data(cfg.paths.processed_data+'patient_data_w_metadata.csv')
patient_df.columns
# patient_df

In [ ]:
patient_df[patient_df['PID'].duplicated()]

In [ ]:
# one_dicom = pydicom.dcmread(cfg.paths.raw_data + patient_df['FolderList'][0][0])
# one_dicom

### Example - CT Images

In [ ]:
temp = load_scan(patient_df['FolderList'][0])

In [ ]:
def browse_images(idx):
    plt.figure(figsize=(5, 5))
    plt.imshow(temp[idx].pixel_array, cmap='gray')
    plt.title(f"Instance: {temp[idx].InstanceNumber} || Total: {len(temp)}")
    plt.axis('off')
    plt.show()

In [ ]:
interact(browse_images, idx=(0, len(temp)-1))

## XML Files

In [ ]:
xml_path = os.path.join(cfg.paths.raw_data, 'calcium_xml')
all_xml_paths = [os.path.join(xml_path, xml) for xml in os.listdir(xml_path) if xml.endswith('.xml')]
patient_xml_dict = {int(os.path.basename(xml_path).split('/')[-1].split('.')[0]): xml_path for xml_path in all_xml_paths}

In [ ]:
for pid, xml_path in patient_xml_dict.items():
    p_xml = load_clean_xml(xml_path)
    if pid == 146:
        break
    xml_list = p_xml['Images']

In [ ]:
xml_list[0]

In [ ]:
patient_df[patient_df['PID']==146]

In [ ]:
with open(xml_path, 'rb') as f:
    plist_data = plistlib.load(f)

print(f"=== Schema View: {os.path.basename(xml_path)} ===\n")
get_plist_schema(plist_data)

In [ ]:
it = iter(main_dict)
for ch in it:
    print(ch.text)

In [ ]:
p_xml